In [0]:
#@title Imports & Set Path
import pandas as pd
from random import randint
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.layers import Dense, CuDNNLSTM, LSTM
from keras.layers import Add, Average 
from keras.layers import Dropout, Activation, Input, Flatten
from keras.layers import TimeDistributed, Bidirectional
from keras import optimizers
from keras import regularizers
from keras.utils.np_utils import to_categorical

from sklearn.model_selection import train_test_split

import warnings

warnings.filterwarnings('ignore')

path = 'Data/' #add the path of your data

In [0]:
#@title reading from drive (if you are using google collaboratory)
from google.colab import drive 
drive.mount('/content/gdrive')

path = 'gdrive/My Drive/challenge-up/'

base = pd.read_csv(path + 'CompleteDataSet_training_competition.csv')
base.loc[base.Tag==20,'Tag'] = 12

In [0]:
#@title Split per second
def split_per_second(dataFrames=None,sensors=None,getLables=True):

  higher_length_sample = 0   
  
  dataFrames_splited_per_second = []

  labels_splited_per_second = np.zeros((1,12))

  cont=0
  current_index=0
  
  dataFrame = dataFrames
  
  teste = dataFrame['TimeStamps'].apply(
      lambda x: x.replace(microsecond=0))
  
  teste = teste.drop_duplicates()
  
  for y in range(len(teste)):
    tDataFrame = dataFrame.iloc[current_index:(current_index+25),:] 
    
    tempDataFrame = tDataFrame.loc[
        tDataFrame['TimeStamps']
        .apply(lambda x: x.replace(microsecond=0))
        .isin(teste.iloc[[y]])
    ]
    
    
    if(tempDataFrame.shape[0] == 0):
      print("ZERO:")
      print(y,teste[y])
      print(tempDataFrame.head())
      continue
        
    
    if higher_length_sample < len(tempDataFrame):
      higher_length_sample = len(tempDataFrame)
      
    current_index+=len(tempDataFrame)
    
    if getLables == True:
      label_mode = tempDataFrame['Tag'].mode()


      if label_mode.shape[0] > 1:
        size=(len(tempDataFrame['Tag'])//2)
        df_new1 = tempDataFrame[:size]
        df_new2 = tempDataFrame[size:]
        
        df_new1 = np.asarray(df_new1.iloc[:,sensors])
        dataFrames_splited_per_second.append(df_new1)
        df_new2 = np.asarray(df_new2.iloc[:,sensors])
        dataFrames_splited_per_second.append(df_new2)
        
        cont+=1
        
                
      else:
        tempDataFrame = np.asarray(tempDataFrame.iloc[:,sensors])
        dataFrames_splited_per_second.append(tempDataFrame)
      
      
      label_mode = np.asarray(label_mode)-1
      label_mode = to_categorical(label_mode,num_classes=12)

      labels_splited_per_second = np.append(
          labels_splited_per_second,label_mode,axis=0)
        
    else:
      tempDataFrame = np.asarray(tempDataFrame.iloc[:,sensors])
      dataFrames_splited_per_second.append(tempDataFrame)
    
    cont+=1
      
  dataFrames_splited_per_second = np.asarray(dataFrames_splited_per_second)
  
  if getLables == True:
    labels_splited_per_second = np.delete(labels_splited_per_second,0,0)
  
  if getLables == True:
    return dataFrames_splited_per_second,labels_splited_per_second,higher_length_sample
  else:
    return dataFrames_splited_per_second, higher_length_sample 

sensors = [
    1,2,3, #accelerometer ankle
    4,5,6, #gyroscope ankle
    #7, #luminosity ankle
    
    8,9,10, #accelerometer right pocket
    11,12,13, #gyroscope right pocket
    #14, #luminosity right pocket
    
    15,16,17, # accelerometer belt
    18,19,20, # gyroscope belt
    #21, #luminosity belt
    
    22,23,24, # accelerometer neck
    25,26,27, # gyroscope neck
    #28, #luminosity neck
    
    #29,30,31, # accelerometer wrist
    #32,33,34, # gyroscope wrist
    ##35, #luminosity wrist
    
    #36 #brain sensor
]
   

base['TimeStamps'] = pd.to_datetime(base['TimeStamps'])
train_dataFrames_splited_per_second,Y_train,higher_length_train = split_per_second(base,sensors,True)

train_dataFrames_splited_per_second,test_dataFrames_splited_per_second, Y_train, Y_test = train_test_split(train_dataFrames_splited_per_second,Y_train,test_size=0.25,random_state=10)
higher_length = higher_length_train


In [0]:
#@title Pad data

def match_data(data=None,index=0,final_len=25):
  
  if(len(data)>final_len):
    print("Error!")
    return(-1)
  
  final_len = final_len-len(data)
  
  for x in range(final_len):
    i = index - x
    vector = data[i]
    vector = np.reshape(vector,(1,vector.shape[0]))
    data = np.concatenate((data[:i],vector,data[i:]),axis=0)

  return data

def completeData(X=None,final_len=22):
  
  new_dataFrames_splited_by_second = []
  for x in range(len(X)):
    test = match_data(
        data=X[x],
        index=len(X[x])-1,
        final_len=higher_length
    )
    new_dataFrames_splited_by_second.append(test)
  
  new_dataFrames_splited_by_second = np.asarray(
      new_dataFrames_splited_by_second)
  
  #print(new_dataFrames_splited_by_second.shape)
  #shape = (len(X),final_len,X[0].shape[1])
  #X = np.reshape(X,shape)
  
  return new_dataFrames_splited_by_second


############################
X_train = completeData(train_dataFrames_splited_per_second,final_len=higher_length)
X_test = completeData(test_dataFrames_splited_per_second,final_len=higher_length)
print("Train:",X_train.shape,Y_train.shape)
print("Test:",X_test.shape,Y_test.shape)
#print(X.shape,Y.shape)

In [0]:
#@title Callback class to print metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class MetricsPrinter(keras.callbacks.Callback):
  def __init__(self):
    self.train_accs = []
    self.train_precisions = []
    self.train_recalls = []
    self.train_f1_scores = []
    self.test_accs = []
    self.test_precisions = []
    self.test_recalls = []
    self.test_f1_scores = []
  
  def on_epoch_end(self, epoch, logs={}):  
    train_predictions = model.predict(X_train)
    y_train_category=[np.argmax(v) for v in Y_train]
    y_train_predict_category=[np.argmax(v) for v in train_predictions]
    
    test_predictions = model.predict(X_test)
    y_test_category=[np.argmax(v) for v in Y_test]
    y_test_predict_category=[np.argmax(v) for v in test_predictions]
    
    # ------
    
    train_acc = accuracy_score(
        y_train_category, y_train_predict_category)
    
    train_precision = precision_score(
        y_train_category, y_train_predict_category, average='micro')
    
    train_recall = recall_score(
        y_train_category, y_train_predict_category, average='micro')
    
    train_f1 = f1_score(
        y_train_category, y_train_predict_category, average='micro')
    
    test_acc = accuracy_score(
        y_test_category, y_test_predict_category)
    
    test_precision = precision_score(
        y_test_category, y_test_predict_category, average='micro')
    
    test_recall = recall_score(
        y_test_category, y_test_predict_category, average='micro')
    
    test_f1 = f1_score(
        y_test_category, y_test_predict_category, average='micro')
    
    # ------
    
    self.train_accs.append(train_acc)
    self.train_precisions.append(train_precision)
    self.train_recalls.append(train_recall)
    self.train_f1_scores.append(train_f1)

    self.test_accs.append(test_acc)
    self.test_precisions.append(test_precision)
    self.test_recalls.append(test_recall)
    self.test_f1_scores.append(test_f1)
    
    # ------
    
    print ('train_acc: %.4f\ttrain_precision: %.4f\t\ttrain_recall: %.4f\ttrain_f1_score: %.4f' 
           % (train_acc, train_precision, train_recall, train_f1))
    
    print ('test_acc: %.4f\ttest_precision: %.4f\t\ttest_recall: %.4f\ttest_f1_score: %.4f' 
           % (test_acc, test_precision, test_recall, test_f1))
    
    print ('----------------------------------------------------------')

In [0]:
#@title Create the model
keras.backend.clear_session()
input_shape = X_train.shape[1:]
print(input_shape)


model = Sequential()


activation_Dense='softmax' 
units = 200

#LSTM MODEL

model.add(Bidirectional(CuDNNLSTM(units=units,
                                  return_sequences=True,
                                  input_shape=input_shape)))

model.add(Dropout(rate=0.25))

model.add(Bidirectional(CuDNNLSTM(units=units,
                                  return_sequences=True)))

model.add(Dropout(rate=0.25))

model.add(Bidirectional(CuDNNLSTM(units=units,
                                  return_sequences=True)))

model.add(Flatten())

model.add(Dense(12,activation=activation_Dense))


adam = optimizers.Adam(lr=0.001, 
                       beta_1=0.9, 
                       beta_2=0.999, 
                       epsilon=None, 
                       decay=0.0, 
                       amsgrad=False)

model.compile(loss = 'categorical_crossentropy',
              optimizer = adam)

metricsPrinter = MetricsPrinter()



In [0]:
#@title Train the model
import matplotlib.pyplot as plt

number_of_epochs = 12 #35

model.fit(
  X_train,
  Y_train,
  epochs=number_of_epochs,
  batch_size=32, 
  verbose=1, 
  shuffle=True,
  callbacks=[metricsPrinter]
)

plt.figure(figsize=(10,5), dpi=100)
plt.grid(True, axis='y', linestyle='--', linewidth=0.5)


plt.plot(metricsPrinter.train_f1_scores, 'bo-', linewidth=3)
plt.plot(metricsPrinter.test_f1_scores, 'ro-', linewidth=3)
plt.ylabel('Percentage')
plt.xlabel('Epoch')
plt.legend(['Train F1-score',
            'Validation F1-score'])

plt.show()
  
  

In [0]:
from sklearn.metrics import confusion_matrix

train_predictions = model.predict(X_train)
y_train_category=[np.argmax(v) for v in Y_train]
y_train_predict_category=[np.argmax(v) for v in train_predictions]
    
test_predictions = model.predict(X_test)
y_test_category=[np.argmax(v) for v in Y_test]
y_test_predict_category=[np.argmax(v) for v in test_predictions]


labels=[1,2,3,4,5,6,7,8,9,10,11,20]

cnf_matrix1 = confusion_matrix(y_train_category, y_train_predict_category)
confusion_ma1 = pd.DataFrame(cnf_matrix1, columns=labels, index=labels)

cnf_matrix2 = confusion_matrix(y_test_category, y_test_predict_category)
confusion_ma2 = pd.DataFrame(cnf_matrix2, columns=labels, index=labels)

In [0]:
#@title Read test set

base_test = pd.read_csv(path + 'CompleteDataSet_testing_competition.csv')

base_test['TimeStamps'] = pd.to_datetime(base_test['TimeStamps'])
base_test = base_test.iloc[1:] 
real_test_splited_per_second, higher_length_real_test = split_per_second(
    base_test,sensors,False)



In [0]:
#@title Generating the final results
X_real_test = completeData(X=real_test_splited_per_second,
                           final_len=higher_length)

print("Test size:",len(X_real_test))
real_test_predictions = model.predict(X_real_test)

y_predict_real=[np.argmax(t) for t in real_test_predictions]

y_predict_real = np.asarray(y_predict_real)+1

y_predict_real = [20 if prediction==12 else prediction for prediction 
                  in y_predict_real]

test_real = base_test['TimeStamps'].apply(
    lambda x: x.replace(microsecond=0))

test_real = test_real.drop_duplicates()

test_real = pd.to_datetime(test_real, 
                           format='%d-%b-%Y %H:%M:%S', 
                           utc=True)

print(test_real.shape[0],len(y_predict_real))

d = {'timestamp': test_real, 'target': y_predict_real}
final_results = pd.DataFrame(data=d)
final_results = final_results[['timestamp','target']]
print(final_results.head(100))


In [0]:
#@title Submission File

!pip install -U -q PyDrive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
import datetime

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
driveWriter = GoogleDrive(gauth)

folder_id = '1-zVMoVGVipYYPDimf1Sw4Gyr8u9oo9_k'
file_title = 'submiss_file_' + str(datetime.datetime.now()) + '.csv' 

submission_file = driveWriter.CreateFile({
    'title': file_title,
    'parents': [{'kind': 'drive#fileLink', 'id': folder_id}]
})

submission_file.SetContentString(
    final_results.to_csv(
        index=False, columns=['timestamp', 'target']
    )
)

submission_file.Upload()